In [1]:
from keras import applications
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras import regularizers
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.utils import layer_utils, np_utils
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [3]:
image_width = 224
image_height = 224

vgg_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))

train_dir = "D:/retinal_samples/basian_opt_testing/new_new_train/augmented_224/train"
test_dir = "D:/retinal_samples/basian_opt_testing/new_test"
    
image_width = 224
image_height = 224
batchsize = 16

In [4]:
def getXTrainYTrainSplit(train_generator):
    x_train, y_train = zip(*(train_generator[i] for i in range(len(train_generator))))
    x_train, y_train = np.vstack(x_train), np.vstack(y_train)
    
    return x_train, y_train

In [65]:
def loadDataKfold(k, train_generator):
    x_train, y_train = zip(*(train_generator[i] for i in range(len(train_generator))))
    x_train = np.vstack(x_train)
    #convert original class labels to the one-hot-encoding
    y_train = np_utils.to_categorical(train_generator.classes, 5)

    #y_train = np.argmax(y_train, axis=1)
    #y_train = np.vstack(map(to_categorical, y_train))[:,0]
    #np.argmax(Y_pred_for_test, axis=1
    folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(x_train, y_train))
    
    return folds, x_train, y_train

In [66]:
def getModel():
    # Create the model
    model = models.Sequential()
    
    # Add the vgg convolutional base model
    model.add(vgg_model)
    
    # Add new layers
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Dense(5, activation='softmax', kernel_regularizer=regularizers.l2(0.0001)))
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-3), metrics=['acc'])
    
    return model

In [67]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_width, image_height),
    batch_size=batchsize,
    shuffle=True,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(image_width, image_height),
        batch_size=batchsize,
        class_mode='categorical',
        shuffle=False)

Found 2000 images belonging to 5 classes.
Found 58 images belonging to 5 classes.


In [42]:
k = 5
folds, x_train, y_train = loadDataKfold(k, train_generator)
model = getModel()
model.summary()

ValueError: bad input shape (125, 16, 5)

In [68]:
def getCallbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, epsilon=1e-4, mode='min')
    
    return [mcp_save, reduce_lr_loss]

In [69]:
x_train, y_train = zip(*(train_generator[i] for i in range(len(train_generator))))
x_train = np.vstack(x_train)
y_train = np_utils.to_categorical(train_generator.classes, 5)
estimator = KerasClassifier(build_fn=getModel, epochs=15, batch_size=batchsize, verbose=0)
#Evaluate The Model with k-Fold Cross Validation
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, x_train, y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

KeyboardInterrupt: 

In [70]:
y_train
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, Y):
  # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X[train], Y[train], epochs=150, batch_size=10, verbose=0)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
for j, (train_idx, val_idx) in enumerate(folds):
    
    print('\nFold ',j)
    x_train_cv = x_train[train_idx]
    y_train_cv = y_train[train_idx]
    x_valid_cv = x_train[val_idx]
    y_valid_cv = y_train[val_idx]
    
    name_weights = "D:/retinal_samples/basian_opt_testing/models/vgg16/final_model_fold" + str(j) + "_weights.h5"
    callbacks = getCallbacks(name_weights = name_weights, patience_lr=10)
    generator = gen.flow(x_train_cv, y_train_cv, batch_size = batch_size)
    model = getModel()
    model.fit_generator(
                generator,
                steps_per_epoch=len(x_train_cv)/batch_size,
                epochs=15,
                shuffle=True,
                verbose=1,
                validation_data = (x_valid_cv, y_valid_cv),
                callbacks = callbacks)
    
    print(model.evaluate(x_valid_cv, y_valid_cv))